<a href="https://colab.research.google.com/github/Sakinat-Folorunso/CMP_805_Advanced_Programming_Languages/blob/main/notebooks/CMP805_Week4_PH_Python_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMP805 — Week 4 Practical (Python, Colab)
**Topic:** Types & Soundness — Simply‑Typed λ‑Calculus (STLC) **type checker**; progress/preservation intuition  
**Course:** Advanced Programming Languages (M.Sc.), OOU — CMP805

**Instructor:** **DR SAKINAT FOLORUNSO – ASSOCIATE PROFESSOR OF AI SYSTEMS AND FAIR DATA**  
**Department:** **COMPUTER SCIENCES, OLABISI ONABANJO UNIVERSITY, AGO‑IWOYE, OGUN STATE, NIGERIA**

> This lab aligns with your Week‑4 plan: **STLC**, soundness ideas, and a **type checker** practical.

### Learning goals (60 minutes)
- Encode **types** and **typing rules** for a core STLC with `Bool`, `Int`, and functions `τ1 → τ2`.
- Implement a **type checker** `type_of(term, Γ)` that reports type errors clearly.
- See how **progress**/**preservation** show up in practice (well‑typed programs don’t get stuck).

In [ ]:
# 🧑‍🎓 Student info
STUDENT_NAME = "Type your full name here"
STUDENT_ID   = "Matric/ID here"
print("Student:", STUDENT_NAME, "| ID:", STUDENT_ID)

In [ ]:
# ✅ Environment check
import sys
major, minor = sys.version_info[:2]
assert (major, minor) >= (3, 10), f"Need Python 3.10+, found {major}.{minor}"
print(f"Python {major}.{minor} OK — match/case available.")

In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, Union, Optional

# ----------------
# Types for STLC
# ----------------
@dataclass(frozen=True) class TBool:  pass
@dataclass(frozen=True) class TInt:   pass
@dataclass(frozen=True) class TFun:   arg: "Type"; ret: "Type"
Type = TBool | TInt | TFun

def show_ty(t: Type) -> str:
    match t:
        case TBool(): return "Bool"
        case TInt():  return "Int"
        case TFun(a,b):
            left = show_ty(a) if not isinstance(a, TFun) else f"({show_ty(a)})"
            return f"{left} -> {show_ty(b)}"

# ----------------
# Terms (STLC + a tiny core for arithmetic/if/let)
# ----------------
@dataclass(frozen=True) class Var:  x: str
@dataclass(frozen=True) class Lam:  x: str; ty: Type; body: "Term"    # λx:τ. body
@dataclass(frozen=True) class App:  f: "Term"; a: "Term"              # f a
@dataclass(frozen=True) class Bool: b: bool
@dataclass(frozen=True) class Int:  n: int
@dataclass(frozen=True) class If:   c: "Term"; t: "Term"; e: "Term"
@dataclass(frozen=True) class Add:  a: "Term"; b: "Term"
@dataclass(frozen=True) class Let:  x: str; e1: "Term"; e2: "Term"    # syntactic sugar, optional

Term = Var | Lam | App | Bool | Int | If | Add | Let

def pretty(e: Term) -> str:
    match e:
        case Var(x):   return x
        case Lam(x,ty,body): return f"\\{x}:{show_ty(ty)}. {pretty(body)}"
        case App(f,a): return f"({pretty(f)} {pretty(a)})"
        case Bool(b):  return str(b).lower()
        case Int(n):   return str(n)
        case If(c,t,e):return f"if {pretty(c)} then {pretty(t)} else {pretty(e)}"
        case Add(a,b): return f"({pretty(a)} + {pretty(b)})"
        case Let(x,e1,e2): return f"let {x} = {pretty(e1)} in {pretty(e2)}"

In [ ]:
# ----------------
# Type checker Γ ⊢ e : τ
# ----------------
class TypeErrorTC(Exception): ...
Context = Dict[str, Type]

def type_of(e: Term, Γ: Optional[Context]=None) -> Type:
    Γ = {} if Γ is None else Γ
    match e:
        case Var(x):
            if x in Γ: return Γ[x]
            raise TypeErrorTC(f"unbound variable {x}")
        case Bool(_):
            return TBool()
        case Int(_):
            return TInt()
        case Add(a,b):
            ta, tb = type_of(a, Γ), type_of(b, Γ)
            if isinstance(ta, TInt) and isinstance(tb, TInt): return TInt()
            raise TypeErrorTC(f"+ expects Int×Int, got {show_ty(ta)} and {show_ty(tb)}")
        case If(c,t,e2):
            tc = type_of(c, Γ)
            if not isinstance(tc, TBool): raise TypeErrorTC(f"if condition must be Bool, got {show_ty(tc)}")
            tt, te = type_of(t, Γ), type_of(e2, Γ)
            if type(tt) != type(te) or (isinstance(tt, TFun) and show_ty(tt)!=show_ty(te)):
                raise TypeErrorTC(f"branches must have same type, got {show_ty(tt)} and {show_ty(te)}")
            return tt
        case Lam(x, ty, body):
            Γ2 = dict(Γ); Γ2[x] = ty
            return TFun(ty, type_of(body, Γ2))
        case App(f, a):
            tf = type_of(f, Γ)
            ta = type_of(a, Γ)
            if isinstance(tf, TFun) and show_ty(tf.arg)==show_ty(ta):
                return tf.ret
            raise TypeErrorTC(f"application expects {show_ty(TFun(ta, TBool()))[:-4]}… but got {show_ty(tf)} applied to {show_ty(ta)}")
        case Let(x, e1, e2):
            tx = type_of(e1, Γ)
            Γ2 = dict(Γ); Γ2[x] = tx
            return type_of(e2, Γ2)
    raise TypeErrorTC("unknown term")

In [ ]:
def check_ok(name: str, e: Term, expect: str):
    ty = show_ty(type_of(e, {}))
    print("ok  -", name, ":", ty)
    assert ty == expect, f"{name}: expected {expect}, got {ty}"

def check_err(name: str, e: Term):
    try:
        type_of(e, {})
        raise AssertionError(f"{name}: expected type error but succeeded")
    except TypeErrorTC as ex:
        print("ok  -", name, "raised:", str(ex).splitlines()[0])

# λx:Int. x + 1 : Int -> Int
id_plus1 = Lam("x", TInt(), Add(Var("x"), Int(1)))
check_ok("lam-add", id_plus1, "Int -> Int")

# (λx:Int. x + 1) 41 : Int
apps = App(id_plus1, Int(41))
check_ok("app-int", apps, "Int")

# if true then 3 else 4 : Int
check_ok("if-int", If(Bool(True), Int(3), Int(4)), "Int")

# let y = 10 in (λx:Int. x + y) 5 : Int
prog = Let("y", Int(10), App(Lam("x", TInt(), Add(Var("x"), Var("y"))), Int(5)))
check_ok("let-lam", prog, "Int")

# Ill-typed: (λx:Int. x) true
bad1 = App(Lam("x", TInt(), Var("x")), Bool(True))
check_err("bad-app", bad1)

# Ill-typed: if 1 then 2 else 3
bad2 = If(Int(1), Int(2), Int(3))
check_err("bad-if", bad2)

### 🧪 Your Turn (10–15 minutes)
1. **Extend the checker**: add a rule for equality `Eq(a,b)` that is well‑typed only when both sides have the **same type** and returns `Bool`.  
2. **Syntactic sugar**: implement `Let` as sugar using λ and application, and show that your type checker gives the **same type** both ways.  
3. **(Optional)** Add a second numeric operator (e.g., `Mul`) and a test.

### ✍️ Reflection (2–3 sentences)
- Where do you see **progress** and **preservation** surface in this implementation?  
- Why do we require an annotation on `λx:τ. e` in **STLC**?